<a href="https://colab.research.google.com/github/dmunozbe/TFM_DataScience/blob/main/Union_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generar .CSV**

La finalidad de este Notebook es la de unir todos los .csv generados en los anteriores Notebooks para poder crear un .csv con todos los indicadores analizados y posteriormente modelizar mediante clustering

In [1]:
# Importamos las librerias necesarias
import csv
import pandas as pd

In [2]:
# Funcion para cargar los .csv
def cargar_csv(ruta):
    try:
        # Cargar el archivo CSV en un DataFrame de pandas
        dataframe = pd.read_csv(ruta)
        return dataframe
    except Exception as e:
        print("Error al cargar el archivo CSV:", e)
        return None

## Dataset de compraventa

### Carga de .csv

In [3]:
df_res = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_RES_Compraventa.csv')
df_res.head()

,Any,Trimestre,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario
0,2022,1,1,Ciutat Vella,1,el Raval,226.1,3203.0
1,2022,1,1,Ciutat Vella,2,el Barri Gòtic,453.0,4755.7
2,2022,1,1,Ciutat Vella,3,la Barceloneta,215.4,4489.8
3,2022,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",353.3,4739.0
4,2022,1,2,Eixample,5,el Fort Pienc,340.0,4175.4


### Adecuación

El dataframe contiene los datos estructurados por año, trimestre, distrito y barrio. Dado que la parte de los indicadores estruturan la informacion por año, mes, distrito y barrio se determina, incluir en el dataframe actual el mes.

In [4]:
# Creamos un DataFrame que contiene todas las combinaciones posibles de año, trimestre y mes
combinaciones = []
for año in df_res['Any'].unique():
    for trimestre in df_res['Trimestre'].unique():
        for mes in range(trimestre * 3 - 2, trimestre * 3 + 1):
            combinaciones.append((año, trimestre, mes))

# Convertimos la lista de combinaciones en un DataFrame
df_completo = pd.DataFrame(combinaciones, columns=['Any', 'Trimestre', 'Mes'])

# Combinamos el DataFrame completo con el DataFrame original
df_completo = pd.merge(df_completo, df_res, on=['Any', 'Trimestre'], how='left')

# Ordenamos el DataFrame resultante
df_completo.sort_values(by=['Any', 'Trimestre', 'Mes'], inplace=True)

# Rellenamos los valores faltantes con los valores del trimestre correspondiente
df_completo['Compraventa_Total'].fillna(method='ffill', inplace=True)  # Propagación hacia adelante
df_completo['Compraventa_Unitario'].fillna(method='ffill', inplace=True)  # Propagación hacia adelante

# Eliminamos las filas con valores nulos
df_completo = df_completo.dropna()

# Modificamos el tipo de dato
df_completo['Codi_Districte'] = df_completo['Codi_Districte'].astype(int)
df_completo['Codi_Barri'] = df_completo['Codi_Barri'].astype(int)

df_completo.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario
7008,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0
7009,2014,1,1,1,Ciutat Vella,2,el Barri Gòtic,380.9,4152.9
7010,2014,1,1,1,Ciutat Vella,3,la Barceloneta,169.6,3927.2
7011,2014,1,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",341.4,3979.2
7012,2014,1,1,2,Eixample,5,el Fort Pienc,309.3,3443.1


In [5]:
# Ordenamos el dataset
df_ordenado = df_completo.sort_values(by=['Any','Codi_Districte','Codi_Barri','Trimestre','Mes'], ascending=[True, True,True,True,True])

In [6]:
df_ordenado.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario
7008,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0
7081,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0
7154,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0
7227,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8
7300,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8


In [7]:
df_ordenado.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8541 entries, 7008 to 8540
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Any                   8541 non-null   int64  
 1   Trimestre             8541 non-null   int64  
 2   Mes                   8541 non-null   int64  
 3   Codi_Districte        8541 non-null   int64  
 4   Nom_Districte         8541 non-null   object 
 5   Codi_Barri            8541 non-null   int64  
 6   Nom_Barri             8541 non-null   object 
 7   Compraventa_Total     8541 non-null   float64
 8   Compraventa_Unitario  8541 non-null   float64
dtypes: float64(2), int64(5), object(2)
memory usage: 667.3+ KB


In [8]:
df_ordenado = df_ordenado.reset_index(drop=True)

In [9]:
df_ordenado.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8


## Indicadores

### Indicador 1. Paro

#### Carga de csv

In [10]:
#Cargamos el csv
df_1 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_1_Paro.csv')
df_1.head()

,Any,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,ParadosReg,NoParados
0,2011,1,1,Ciutat Vella,1,el Raval,5034,0.0
1,2011,1,1,Ciutat Vella,2,el Barri Gòtic,1468,0.0
2,2011,1,1,Ciutat Vella,3,la Barceloneta,1463,0.0
3,2011,1,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",2023,0.0
4,2011,1,2,Eixample,5,el Fort Pienc,1901,0.0


In [11]:
#Eliminamos las columnas de texto
df_1 = df_1.drop(columns=['Nom_Districte', 'Nom_Barri'])

#### Merge con dataset compraventa

In [12]:
# Unimos el dataframe df_1 al df_ordenado
df_ordenado_df1 = pd.merge(df_ordenado, df_1, left_on=['Any','Mes','Codi_Districte','Codi_Barri'], right_on=['Any','Mes','Codi_Districte','Codi_Barri'], how='left')
df_ordenado_df1.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,NoParados
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,0.0
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,0.0
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,0.0
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,0.0
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,0.0


In [13]:
df_ordenado_df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Any                   8541 non-null   int64  
 1   Trimestre             8541 non-null   int64  
 2   Mes                   8541 non-null   int64  
 3   Codi_Districte        8541 non-null   int64  
 4   Nom_Districte         8541 non-null   object 
 5   Codi_Barri            8541 non-null   int64  
 6   Nom_Barri             8541 non-null   object 
 7   Compraventa_Total     8541 non-null   float64
 8   Compraventa_Unitario  8541 non-null   float64
 9   ParadosReg            8541 non-null   int64  
 10  NoParados             8541 non-null   float64
dtypes: float64(3), int64(6), object(2)
memory usage: 734.1+ KB


### Indicador 10. Estudios

#### Carga de csv

In [14]:
# Cargamos el csv
df_10 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_10_Estudios.csv')
df_10.head()

,Codi_Districte,Nom_Districte,Any,Sin_Estudios,Solo_Estudios_Obligatorios,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios
0,1,Ciutat Vella,2016,2.8,46.5,20.4,29.0,97.2
1,1,Ciutat Vella,2017,2.4,45.5,20.2,30.6,97.6
2,1,Ciutat Vella,2018,2.1,44.9,20.1,31.6,97.9
3,1,Ciutat Vella,2019,1.8,43.9,20.6,32.3,98.2
4,1,Ciutat Vella,2020,1.6,44.4,20.8,32.1,98.4


In [15]:
#Eliminamos las columnas de texto
df_10 = df_10.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [16]:
# Unimos el dataframe df_10 al df_ordenado_df1
df_ordenado_df10 = pd.merge(df_ordenado_df1, df_10, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df10.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,NoParados,Sin_Estudios,Solo_Estudios_Obligatorios,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,0.0,NaN,NaN,NaN,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,0.0,NaN,NaN,NaN,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,0.0,NaN,NaN,NaN,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,0.0,NaN,NaN,NaN,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,0.0,NaN,NaN,NaN,NaN,NaN


In [17]:
df_ordenado_df10.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 16 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte               8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 15. Energia

#### Carga de csv

In [18]:
# Cargamos el csv
df_15 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_15_Energia.csv')
df_15.head()

,Any,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Codi_Postal,ConsumIndustria,ConsumResidencial,ConsumServeis
0,2019,1,1,Ciutat Vella,1,el Raval,8001,187412,5547299,8695319
1,2019,1,1,Ciutat Vella,1,el Raval,8002,472527,4535282,15036860
2,2019,1,1,Ciutat Vella,1,el Raval,8007,483444,3115277,9333701
3,2019,1,1,Ciutat Vella,1,el Raval,8038,7667787,4812237,7547921
4,2019,1,1,Ciutat Vella,1,el Raval,8039,8116124,818276,8282498


In [19]:
# Agrupamos los datos por cogido postal
df_15 = df_15.groupby(['Any', 'Mes','Codi_Districte','Nom_Districte','Codi_Barri','Nom_Barri'])[['ConsumIndustria','ConsumResidencial','ConsumServeis']].sum()

In [20]:
df_15 = df_15.reset_index()

In [21]:
#Eliminamos las columnas de texto
df_15 = df_15.drop(columns=['Nom_Districte', 'Nom_Barri'])

#### Merge con dataset compraventa

In [22]:
df_ordenado_df15 = pd.merge(df_ordenado_df10, df_15, left_on=['Any','Mes','Codi_Districte','Codi_Barri'], right_on=['Any','Mes','Codi_Districte','Codi_Barri'], how='left')
df_ordenado_df15.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,NoParados,Sin_Estudios,Solo_Estudios_Obligatorios,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_ordenado_df15.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 19 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte               8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 22. PM2.5

#### Carga de csv

In [24]:
# Cargamos el csv
df_22 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_22_PM25.csv')
df_22.head()

,Any,Codi_Districte,Nom_Districte,Pm25
0,2018,1,Ciutat Vella,2.282272
1,2018,2,Eixample,2.990421
2,2018,3,Sants-MontjuÃ¯c,2.213685
3,2018,4,Les Corts,2.459279
4,2018,5,SarriÃ -Sant Gervasi,2.327853


In [25]:
#Eliminamos las columnas de texto
df_22 = df_22.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [26]:
df_ordenado_df22 = pd.merge(df_ordenado_df15, df_22, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df22.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,NoParados,Sin_Estudios,Solo_Estudios_Obligatorios,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
df_ordenado_df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte               8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 23. PM10

#### Carga de csv

In [28]:
# Cargamos el csv
df_23 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_23_PM10.csv')
df_23.head()

,Any,Codi_Districte,Nom_Districte,Pm10
0,2018,1,Ciutat Vella,2.018359
1,2018,2,Eixample,2.054598
2,2018,3,Sants-MontjuÃ¯c,2.007203
3,2018,4,Les Corts,1.974633
4,2018,5,SarriÃ -Sant Gervasi,1.795715


In [29]:
#Eliminamos las columnas de texto
df_23 = df_23.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [30]:
df_ordenado_df23 = pd.merge(df_ordenado_df22, df_23, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df23.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Sin_Estudios,Solo_Estudios_Obligatorios,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25,Pm10
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
df_ordenado_df23.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 21 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte               8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 40. Participacion

#### Carga de csv

In [32]:
# Cargamos el csv
df_40 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_40_Participacion.csv')
df_40.head()

,Codi_Districte,Nom_Districte,Any,Valor
0,1,Ciutat Vella,1987,62.5
1,1,Ciutat Vella,1991,44.2
2,1,Ciutat Vella,1995,48.0
3,1,Ciutat Vella,1999,44.0
4,1,Ciutat Vella,2003,48.0


In [33]:
df_40 = df_40.rename(columns={'Valor': 'Participacion'})

In [34]:
# Convertir la columna 'Any' en el índice
df_40.set_index('Any', inplace=True)

In [35]:
# Reindexar con años continuos y llenar los valores faltantes con el último valor conocido
df_filled = df_40.groupby('Codi_Districte').apply(lambda group: group.reindex(range(group.index.min(), group.index.max() + 1), method='ffill'))

In [36]:
df_filled.head()

Codi_Districte Nom_Districte  Participacion
Codi_Districte Any                                              
1              1987               1  Ciutat Vella           62.5
               1988               1  Ciutat Vella           62.5
               1989               1  Ciutat Vella           62.5
               1990               1  Ciutat Vella           62.5
               1991               1  Ciutat Vella           44.2

In [37]:
# Restablecer el índice y eliminar la columna 'Nom_Districte' duplicada
df_filled.drop(columns='Codi_Districte', inplace=True)
print("\nDataFrame con valores del último año conocido:")
df_filled.reset_index()


DataFrame con valores del último año conocido:


,Codi_Districte,Any,Nom_Districte,Participacion
0,1,1987,Ciutat Vella,62.5
1,1,1988,Ciutat Vella,62.5
2,1,1989,Ciutat Vella,62.5
3,1,1990,Ciutat Vella,62.5
4,1,1991,Ciutat Vella,44.2
...,...,...,...,...
365,10,2019,Sant Martí,66.5
366,10,2020,Sant Martí,66.5
367,10,2021,Sant Martí,66.5
368,10,2022,Sant Martí,66.5


#### Merge con dataset compraventa

In [38]:
df_ordenado_df40 = pd.merge(df_ordenado_df23, df_filled, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df40.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Bachillerato,Estudios_Universitarios,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25,Pm10,Nom_Districte_y,Participacion
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8


In [39]:
df_ordenado_df40.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 46. Esperanza de vida

#### Carga de csv

In [40]:
#Cargamos los diferentes datasets, en diferentes dataframes
df_46 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_46_EsperanzaVida.csv')
df_46.head()

,Anys_Periode,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,AñosEspVida,Any
0,2006-2010,1,Ciutat Vella,1,el Raval,82.6,2010
1,2006-2010,1,Ciutat Vella,2,el Barri Gòtic,84.7,2010
2,2006-2010,1,Ciutat Vella,3,la Barceloneta,83.3,2010
3,2006-2010,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",85.1,2010
4,2006-2010,2,Eixample,5,el Fort Pienc,86.7,2010


In [41]:
#Eliminamos las columnas de texto
df_46 = df_46.drop(columns=['Nom_Districte','Anys_Periode','Nom_Barri'])

#### Merge con dataset compraventa

In [42]:
df_ordenado_df46 = pd.merge(df_ordenado_df40, df_46, left_on=['Any','Codi_Districte','Codi_Barri'], right_on=['Any','Codi_Districte','Codi_Barri'], how='left')
df_ordenado_df46.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Estudios_Universitarios,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25,Pm10,Nom_Districte_y,Participacion,AñosEspVida
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7


In [43]:
df_ordenado_df46.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 24 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 73. Internet

#### Carga de csv

In [44]:
# Cargamos el csv
df_73 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_73_Internet.csv')
df_73.head()

,Codi_Districte,Nom_Districte,Any,Internet
0,1,Ciutat Vella,2016,82.8
1,1,Ciutat Vella,2017,85.5
2,1,Ciutat Vella,2018,90.9
3,1,Ciutat Vella,2019,80.4
4,1,Ciutat Vella,2020,90.4


In [45]:
#Eliminamos las columnas de texto
df_73 = df_73.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [46]:
df_ordenado_df73 = pd.merge(df_ordenado_df46, df_73, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df73.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Estudios_Obligatorios,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25,Pm10,Nom_Districte_y,Participacion,AñosEspVida,Internet
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN


In [47]:
df_ordenado_df73.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 25 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 74. Movil

#### Carga de csv

In [48]:
# Cargamos el csv
df_74 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_74_Movil.csv')
df_74.head()

,Codi_Districte,Nom_Districte,Any,Movil
0,1,Ciutat Vella,2016,96.3
1,1,Ciutat Vella,2017,96.6
2,1,Ciutat Vella,2018,99.0
3,1,Ciutat Vella,2019,98.8
4,1,Ciutat Vella,2020,98.5


In [49]:
#Eliminamos las columnas de texto
df_74 = df_74.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [50]:
df_ordenado_df74 = pd.merge(df_ordenado_df73, df_74, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df74.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,ConsumIndustria,ConsumResidencial,ConsumServeis,Pm25,Pm10,Nom_Districte_y,Participacion,AñosEspVida,Internet,Movil
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,Ciutat Vella,41.8,80.7,NaN,NaN


In [51]:
df_ordenado_df74.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 26 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 79. Indice de Motorización

#### Carga de csv

In [52]:
# Cargamos el csv
df_79 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_79_IndeceMotorizacion.csv')
df_79.head()

,Any,Codi_Districte,Nom_Districte,Codi_Barri,Nom_Barri,Index_Turisme,Index_Motos,Index_Ciclomotors,Index_Furgonetes,Index_Camions,Index_Resta
0,2019,1,Ciutat Vella,1,el Raval,122.718571,64.096190,27.518571,18.271905,6.926667,4.759048
1,2019,1,Ciutat Vella,2,el Gòtic,334.384444,212.484444,29.957778,101.367778,66.910000,26.407778
2,2019,1,Ciutat Vella,3,la Barceloneta,208.480909,117.584545,33.890000,29.419091,6.974545,5.944545
3,2019,1,Ciutat Vella,4,"Sant Pere, Santa Caterina i la Ribera",167.408462,110.965385,231.558462,22.277692,5.622308,5.066154
4,2019,2,Eixample,5,el Fort Pienc,277.481000,124.800000,25.181500,22.122500,8.354000,12.688000


In [53]:
#Eliminamos las columnas de texto
df_79 = df_79.drop(columns=['Nom_Districte','Nom_Barri'])

#### Merge con dataset compraventa

In [54]:
df_ordenado_df79 = pd.merge(df_ordenado_df74, df_79, left_on=['Any','Codi_Districte','Codi_Barri'], right_on=['Any','Codi_Districte','Codi_Barri'], how='left')
df_ordenado_df79.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Participacion,AñosEspVida,Internet,Movil,Index_Turisme,Index_Motos,Index_Ciclomotors,Index_Furgonetes,Index_Camions,Index_Resta
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,41.8,80.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,41.8,80.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,41.8,80.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,41.8,80.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,41.8,80.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
df_ordenado_df79.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 32 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

### Indicador 85. Zonas Verdes

#### Carga de csv

In [56]:
# Cargamos el csv
df_85 = cargar_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/datos_85_ZonasVerdes.csv')
df_85.head()

,Codi_Districte,Nom_Districte,Any,Zona Verde (m²),m² por habitante
0,1,CiutatVella,2000,623200,7.4
1,1,CiutatVella,2001,624300,7.4
2,1,CiutatVella,2002,624100,6.4
3,1,CiutatVella,2003,625100,5.9
4,1,CiutatVella,2004,625100,5.8


In [57]:
#Eliminamos las columnas de texto
df_85 = df_85.drop(columns=['Nom_Districte'])

#### Merge con dataset compraventa

In [58]:
df_ordenado_df85 = pd.merge(df_ordenado_df79, df_85, left_on=['Any','Codi_Districte'], right_on=['Any','Codi_Districte'], how='left')
df_ordenado_df85.head()

,Any,Trimestre,Mes,Codi_Districte,Nom_Districte_x,Codi_Barri,Nom_Barri,Compraventa_Total,Compraventa_Unitario,ParadosReg,...,Internet,Movil,Index_Turisme,Index_Motos,Index_Ciclomotors,Index_Furgonetes,Index_Camions,Index_Resta,Zona Verde (m²),m² por habitante
0,2014,1,1,1,Ciutat Vella,1,el Raval,142.8,2813.0,4588,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628919.0,6.2
1,2014,1,2,1,Ciutat Vella,1,el Raval,142.8,2813.0,4539,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628919.0,6.2
2,2014,1,3,1,Ciutat Vella,1,el Raval,142.8,2813.0,4572,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628919.0,6.2
3,2014,2,4,1,Ciutat Vella,1,el Raval,195.2,2948.8,4510,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628919.0,6.2
4,2014,2,5,1,Ciutat Vella,1,el Raval,195.2,2948.8,4379,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,628919.0,6.2


In [59]:
df_ordenado_df85.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8541 entries, 0 to 8540
Data columns (total 34 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Any                         8541 non-null   int64  
 1   Trimestre                   8541 non-null   int64  
 2   Mes                         8541 non-null   int64  
 3   Codi_Districte              8541 non-null   int64  
 4   Nom_Districte_x             8541 non-null   object 
 5   Codi_Barri                  8541 non-null   int64  
 6   Nom_Barri                   8541 non-null   object 
 7   Compraventa_Total           8541 non-null   float64
 8   Compraventa_Unitario        8541 non-null   float64
 9   ParadosReg                  8541 non-null   int64  
 10  NoParados                   8541 non-null   float64
 11  Sin_Estudios                6132 non-null   float64
 12  Solo_Estudios_Obligatorios  6132 non-null   float64
 13  Bachillerato                6132 

In [60]:
df_ordenado_df85.to_csv('/content/drive/MyDrive/UOC/TFM/Datasets definitivos/Final.csv', index=False)